In [4]:
from asf_datasets.lmdb_data import LMDBDataset
from torch_geometric.loader import DataLoader
train_set = LMDBDataset(lmdb_path='/ve/junfu/AtomScaffold2/data/mptraj',split='train',label_keys=['energy_per_atom','forces','stress'])
data_loader = DataLoader(train_set, batch_size=3, shuffle=True, num_workers=0, pin_memory=True)
data = next(iter(data_loader))

/ve/junfu/miniconda3/envs/asf/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /usr/lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /ve/junfu/miniconda3/envs/asf/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/ve/junfu/miniconda3/envs/asf/lib/python3.12/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /ve/junfu/miniconda3/envs/asf/lib/python3.12/site-packages/torch_cluster/_version_cuda.so)
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/ve/junfu/miniconda3/envs/asf/lib/python3.12/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usa

read length:1559916
available keys:dict_keys(['numbers', 'positions', 'forces', 'cell', 'pbc', 'energy', 'stress', 'e_per_atom_relaxed', 'mp_id', 'energy_per_atom', 'ef_per_atom_relaxed', 'corrected_total_energy', 'ef_per_atom', 'task_id', 'calc_id', 'ionic_step', 'filename', 'extxyz_id', 'num_atoms', 'corrected_total_energy_relaxed', 'energy_referenced', 'corrected_total_energy_referenced', 'corrected_total_energy_relaxed_referenced', 'composition'])
used keys:['energy_per_atom', 'forces', 'stress']


In [ ]:
from importlib import reload
from asf_models.cet import cet
reload(cet)

# class Args: 
#     max_atoms_number = 128
#     atom_embedding_dim = 128
#     edge_embedding_dim = 128
#     num_basis = 16
#     cutoff = 5.0
#     lmax = 2
#     n_heads = 4
#     num_blocks = 2

class Args: 
    max_numbers_of_atoms = 128
    embedding_dim = 128
    cutoff = 5.0
    num_basis = 16
    lmax = 2
    
    
args = Args()
model = cet.Cet(args)

out = model(data)
out.shape

AttributeError: 'Args' object has no attribute 'lmax'

In [49]:
out['expand_mask'].shape

torch.Size([3, 157])